# The Revenge of Rocchio's Angels

We will be #1 this time

Local Script Dependencies

In [ ]:
!python -c "import platform; print(platform.platform()); import sys; print(sys.version)"

In [1]:
from engine import SearchEngine
from evaluate_map import *
from optimizing import Optimize
# from ebrm import *

C:\Users\ariel\PycharmProjects\FinalProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
se = SearchEngine()

In [4]:
se.set_searcher(approach="qld")

In [ ]:
results = se.get_top_k("international organized crime", k=50)

In [ ]:
results[0][0][:300]

In [3]:
topics = load_topics("Data/queriesROBUST.txt")

In [6]:
# This one creates a file called "run.txt" with submitting format, can change file name
se.search_all_queries(topics, 100)

In [7]:
qrels = load_qrels("Data/qrels_50_Queries")   # or "qrel301.txt"
run   = load_run("run.txt")

map_score, ap_by_q = mean_average_precision(qrels, run)
map_score

0.24594939993995996

In [7]:
map = get_map_by_paths("Data/qrels_50_Queries", "run.txt")

In [8]:
map

0.21283191880598026

In [4]:
opti = Optimize()
# fb_terms_values = [5, 6, 8, 10, 15, 20]
# fb_docs_values = [5, 7, 10, 15]
# og_query_weight_values = [0.3, 0.4, 0.5, 0.6, 0.7]
mus = [250 + i for i in range(10,100,5)]

opti.optimize_qld(topics, [20], [5], [0.6], mus, k=1000)

fb_terms=20, fb_docs=5, w=0.6, mu=260 -> MAP=0.262942
fb_terms=20, fb_docs=5, w=0.6, mu=265 -> MAP=0.262857
fb_terms=20, fb_docs=5, w=0.6, mu=270 -> MAP=0.264357
fb_terms=20, fb_docs=5, w=0.6, mu=275 -> MAP=0.264102
fb_terms=20, fb_docs=5, w=0.6, mu=280 -> MAP=0.264134
fb_terms=20, fb_docs=5, w=0.6, mu=285 -> MAP=0.263871
fb_terms=20, fb_docs=5, w=0.6, mu=290 -> MAP=0.264126
fb_terms=20, fb_docs=5, w=0.6, mu=295 -> MAP=0.264063
fb_terms=20, fb_docs=5, w=0.6, mu=300 -> MAP=0.264891
fb_terms=20, fb_docs=5, w=0.6, mu=305 -> MAP=0.264674
fb_terms=20, fb_docs=5, w=0.6, mu=310 -> MAP=0.265222
fb_terms=20, fb_docs=5, w=0.6, mu=315 -> MAP=0.265174
fb_terms=20, fb_docs=5, w=0.6, mu=320 -> MAP=0.265064
fb_terms=20, fb_docs=5, w=0.6, mu=325 -> MAP=0.264215
fb_terms=20, fb_docs=5, w=0.6, mu=330 -> MAP=0.264509
fb_terms=20, fb_docs=5, w=0.6, mu=335 -> MAP=0.263725
fb_terms=20, fb_docs=5, w=0.6, mu=340 -> MAP=0.265555
fb_terms=20, fb_docs=5, w=0.6, mu=345 -> MAP=0.265381

BEST:
fb_terms=20, fb_docs=

In [ ]:
from ebrm import extract_entities_to_jsonl

summary = extract_entities_to_jsonl(
    index_name="robust04",
    out_path="robust04_entities.jsonl",
    batch_size=256,   # 128 if you get memory issues
    n_process=1,      # on Windows notebooks, keep 1 (increase only if it's stable)
    limit_docs=None,  # FULL RUN
    progress_every=5000,
    drop_leading_det=True,
    min_entity_len=3,
)

summary

C:\Users\ariel\PycharmProjects\FinalProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[entity_extraction] wrote 5000 docs...
[entity_extraction] wrote 10000 docs...


In [3]:
from collect_candidates_docids import collect_candidate_docids

docids_path = collect_candidate_docids(
    se.searcher,
    queries_path="Data/queriesROBUST.txt",
    out_path="candidate_docids_top200_rm3best.txt",
    approach="qld",
    mu=340,
    rm3=False,
    fb_terms=20,
    fb_docs=5,
    original_query_weight=0.6,
    topk_per_query=8
)
docids_path


[collect_docids] 25/249 queries, unique docids=199
[collect_docids] 50/249 queries, unique docids=397
[collect_docids] 75/249 queries, unique docids=593
[collect_docids] 100/249 queries, unique docids=791
[collect_docids] 125/249 queries, unique docids=981
[collect_docids] 150/249 queries, unique docids=1172
[collect_docids] 175/249 queries, unique docids=1359
[collect_docids] 200/249 queries, unique docids=1556
[collect_docids] 225/249 queries, unique docids=1755
[collect_docids] wrote 1938 docids to candidate_docids_top200_rm3best.txt


'candidate_docids_top200_rm3best.txt'

In [4]:
from entity_extraction_subset import extract_entities_for_docids_to_jsonl

summary = extract_entities_for_docids_to_jsonl(
    docids_path="candidate_docids_top200_rm3best.txt",
    out_path="robust04_entities_subset.jsonl",
    batch_size=256,
    n_process=1
)
summary

{'out_path': 'C:\\Users\\ariel\\PycharmProjects\\FinalProject\\robust04_entities_subset.jsonl',
 'docids_input': 1938,
 'docs_written': 1938,
 'total_entities': 41490,
 'avg_entities_per_doc': 21.408668730650156,
 'batch_size': 256,
 'n_process': 1,
 'labels': ['GPE', 'LOC', 'ORG', 'PERSON']}

In [5]:
from build_offsets import build_offsets
offsets_path = build_offsets("robust04_entities_subset.jsonl")
offsets_path

'robust04_entities_subset.jsonl.offsets.pkl'

In [4]:
from engine import SearchEngine

se = SearchEngine()
se.set_searcher(approach="bm25", fb_terms=20, fb_docs=5, original_query_weight=0.6, mu=340,k1=0.5,b=0.36)
se.search_all_queries(topics, k=1000, run_tag="rm3_best", output_file="run_bm25rm3_best.txt")
map = get_map_by_paths("Data/qrels_50_Queries", "run_bm25rm3_best.txt")
map

0.2721085149407856

In [4]:
se = SearchEngine(
    entities_jsonl="robust04_entities_subset.jsonl",
    entities_offsets="robust04_entities_subset.jsonl.offsets.pkl"
)

se.set_searcher(approach="qld", fb_terms=20, fb_docs=5, original_query_weight=0.6, mu=340)

se.search_all_queries_hybrid_ebrm(
    topics=topics,
    k=1000,
    run_tag="rm3_best_plus_ebrm",
    output_file="run_hybrid_ebrm.txt",
    fb_docs=5,
    fb_entities=10,
    entity_weight=0.1,
    allowed_labels={"PERSON", "ORG", "GPE", "LOC"}
)


international organized crime "Ministry of Internal Affairs"^0.061019 BFN^0.042408 "B. Yeltsin"^0.042408 "Russian Federation Federal Program"^0.042408 "Russian Federation General Prosecutor's Office"^0.042408 "Russian Federation"^0.041701 Moscow^0.041701 MVD^0.040260 Russia^0.039553 Fight^0.021466
poliomyelitis post polio "United States"^0.038901 CSO^0.033634 Gujarat^0.033634 "World Health Organisation"^0.033634 "Shaktisinh Gohil"^0.033634 "Kumar Nathani"^0.033634 Nathani^0.033634 Yamini^0.033634 "Bharat Bhagat"^0.033634 "Polio Foundation"^0.033634
hubble telescope achievements Hubble^0.083589 Earth^0.066354 NASA^0.061381 "National Aeronautics and Space Administration"^0.044970 "Perkin-Elmer Corp."^0.044970 "Jet Propulsion Laboratory"^0.044970 Truly^0.029074 "Hughes Danbury Optical Systems of Danbury"^0.029074 "Conn. Company"^0.029074 "Lew Allen"^0.029074
endangered species mammals "Marine Mammal Protection Act"^0.061910 "National Marine Fisheries Service"^0.061910 "National Oceanic an

KeyboardInterrupt: 

In [9]:
map = get_map_by_paths("Data/qrels_50_Queries", "run_hybrid_ebrm.txt")


In [10]:
map

0.2655553852200375